# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

In [2]:
from sqlalchemy import create_engine

In [3]:
# load data from database
engine = create_engine('sqlite:///DataBase.db')
df = pd.read_sql('SELECT * FROM df', engine)

In [ ]:
df.head(3)

In [ ]:
df.genre.unique()

#### cleaning non-binary columns:

In [13]:
df.related.value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [14]:
bad_samples = []
for i in df.index:
    if df.loc[i,'related'] == 2:
        bad_samples.append(i)

In [16]:
len(bad_samples)

188

In [18]:
good_samples = [i for i in df.index if i not in bad_samples ]

In [20]:
# checking:
len(good_samples)+len(bad_samples), df.shape[0]

(26216, 26216)

In [22]:
df = df.iloc[good_samples,]
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [24]:
# final check:
df.related.value_counts()

1    19906
0     6122
Name: related, dtype: int64

#### splitting  feature vs target:

In [25]:
features = ['message']

In [26]:
X = df[features]
X.head(3)

,message
0,Weather update - a cold front from Cuba that c...
1,Is the Hurricane over or is it not over
2,Looking for someone but no name


In [27]:
non_target_cols = ['id','message','original','genre'] 
non_target_cols

['id', 'message', 'original', 'genre']

In [28]:
target_cols = df.columns[~df.columns.isin(non_target_cols)]
target_cols

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [29]:
Y = df[target_cols]
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [30]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [32]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

In [34]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize) ),
    ('tfidf', TfidfTransformer() ),
    ('clf', MultiOutputClassifier(RandomForestClassifier() ) )
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [35]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X.message,Y)

In [42]:
(X_train.shape , Y_train.shape)

((19521,), (19521, 36))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [38]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [39]:
from sklearn.multioutput import MultiOutputClassifier

In [43]:
clf = pipeline.fit(X_train, Y_train)

In [44]:
Y_pred = pipeline.predict(X_test)

In [ ]:
(Y_test.shape, Y_pred.shape)

In [ ]:
[type(Y_test), type(Y_pred)]

In [45]:
Y_pred = pd.DataFrame(Y_pred,columns=Y_test.columns)
Y_pred.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import numpy as np

In [46]:
from sklearn.utils.multiclass import type_of_target

In [ ]:
[type_of_target(Y_test), type_of_target(Y_pred)]

In [47]:
print(classification_report(Y_test,Y_pred,target_names=Y_test.columns))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      4976
               request       0.80      0.43      0.56      1150
                 offer       0.00      0.00      0.00        31
           aid_related       0.73      0.61      0.66      2725
          medical_help       0.54      0.11      0.19       532
      medical_products       0.79      0.13      0.22       330
     search_and_rescue       0.53      0.10      0.17       169
              security       0.00      0.00      0.00       123
              military       0.50      0.08      0.14       218
           child_alone       0.00      0.00      0.00         0
                 water       0.80      0.34      0.48       395
                  food       0.83      0.49      0.62       722
               shelter       0.79      0.36      0.49       565
              clothing       0.71      0.17      0.27       103
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [49]:
print(f1_score((Y_test), (Y_pred),average='macro'))

0.247663579621


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.